Unique Values

In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv(r"D:\\GUVI AI & ML\\Machine-Learning\\Streamlit_house_prediction\\House_Data.csv")
df

In [5]:
# Identify categorical columns and exclude date columns
# Assuming that date columns are those which have date in their name
categorical_columns = [col for col in df.columns if df[col].dtype == 'object' and 'date' not in col.lower()]

# create a dictionary
unique_dict = {}

for col in categorical_columns:
    unique_values = df[col].unique()
    unique_dict[col] = unique_values

# create Dataframe to store unique values

# use a dataframe with each column containing unique values of corressponding categorical columns

unique_values_df = pd.DataFrame(dict([(k,pd.Series(v)) for k, v in unique_dict.items()]))

# Save the Dataframe to a csv file
output_file_path = r"D:\\GUVI AI & ML\\Machine-Learning\\Streamlit_house_prediction\\categorical_values.csv"
unique_values_df.to_csv(output_file_path, index=False)

print("Unique values for categorical columns have been save to", output_file_path)

Unique values for categorical columns have been save to D:\\GUVI AI & ML\\Machine-Learning\\Streamlit_house_prediction\\categorical_values.csv


Correcting The Spelling Mistakes

In [8]:
import pandas as pd

# define corrections for categorical columns

corrections = {
    'AREA':{
        'Karapakkam':'Karapakkam',
        'Chrompt':'Chrompet',
        'Chrmpet':'Chrompet',
        'Ana Nagar':'Anna Nagar',
        'Adyr':'Adyar',
        'Velchery':'Velachery',
        'Ann Nagar':'Anna Nagar',
        'KKNagar':'KK Nagar',
        'Karapakam':'Karapakkam',
        'Chormpet':'Chrompet'
    },
    'SALE_COND':{
      'AbNormal':'Ab Normal',
      'Ab Normal':'Ab Normal',
      'Partiall':'Partial',
      'Adj Land' : 'AdjLand',
      'PartiaLl' : 'Partial'
    },
    'PARK_FACIL':{
        'Noo':'No'
    },
    'BUILDTYPE':{
        'Commercial':'Commercial',
        'Other':'Others'
    },
    'UTILITY_AVAIL':{
        'ELO':'AllPub',
        'NoSewr':'NoSewa',
        'NoSeWa':'NoSeWa',
        'All Pub':'AllPub'
    },
    'STREET':{
        'Pavd':'Paved',
        'NoAccess':'No Access'
    }
}

# load the dataset
 
file_path = r"D:\\GUVI AI & ML\\Machine-Learning\\Streamlit_house_prediction\\House_Data.csv"
house_data = pd.read_csv(file_path)

# Apply correction to categorical columns

for col, mapping in corrections.items():
    if col in house_data.columns:
        house_data[col] = house_data[col].replace(mapping)


# Save the corrected data to a new CSV file
corrected_file_path = r"D:\\GUVI AI & ML\\Machine-Learning\\Streamlit_house_prediction\\House_Sales_data_corrected.csv"
house_data.to_csv(corrected_file_path,index=False)

print("Corrected file is Saved")

Corrected file is Saved


Data Preprocessing

In [14]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import joblib


# Load Dataset
file_path = r"D:\\GUVI AI & ML\\Machine-Learning\\Streamlit_house_prediction\\House_Sales_data_corrected.csv"
house_data = pd.read_csv(file_path)

# convert date columns to datetime format

house_data['DATE_SALE'] = pd.to_datetime(house_data['DATE_SALE'],format='%d-%m-%Y')
house_data['DATE_BUILD'] = pd.to_datetime(house_data['DATE_BUILD'],format='%d-%m-%Y')

# Extract useful components from date columns
house_data['SALE_YEAR']  = house_data['DATE_SALE'].dt.year
house_data['SALE_MONTH']  = house_data['DATE_SALE'].dt.month
house_data['BUILD_YEAR']  = house_data['DATE_BUILD'].dt.year
house_data['BUILD_AGE']  = house_data['SALE_YEAR'] - house_data['BUILD_YEAR']


# Drop original data columns if they are no longer needed

house_data = house_data.drop(columns=['DATE_SALE','DATE_BUILD'])

# Drop features that are less likely to help in prediction or may lead to overfitting
# for instance,identifiers and features with too many categories or minimal variance

house_data = house_data.drop(columns=['PRT_ID','SALE_COND','STREET','MZZONE','UTILITY_AVAIL'])

# Handle Missing Values

num_cols = house_data.select_dtypes(include=['float64','int64']).columns
imputer = SimpleImputer(strategy='mean')
house_data[num_cols]=imputer.fit_transform(house_data[num_cols])

# Encode Categorical variables
cat_cols = house_data.select_dtypes(include=['object']).columns
label_encoder = {}
for col in cat_cols:
    le = LabelEncoder()
    house_data[col] = le.fit_transform(house_data[col])
    label_encoder[col] = le

# Save label encoders as a joblib file
label_encoder_file_path = r"D:\\GUVI AI & ML\\Machine-Learning\\Streamlit_house_prediction\\label_encoders.pkl"
joblib.dump(label_encoder,label_encoder_file_path)


# Save preprocessed data
output_file_path = r"D:\\GUVI AI & ML\\Machine-Learning\\Streamlit_house_prediction\\preprocessed_house_data.csv"
house_data.to_csv(output_file_path,index=False)

print("Preprocessed file and label encoders are saved")

Preprocessed file and label encoders are saved


Feature Selection

In [15]:
import pandas as pd
from sklearn.model_selection  import train_test_split
from sklearn.ensemble import RandomForestRegressor
import joblib

# Load Data
house_data = pd.read_csv(r"D:\\GUVI AI & ML\\Machine-Learning\\Streamlit_house_prediction\\preprocessed_house_data.csv")

# Define features and target variables

X = house_data.drop(columns=['SALES_PRICE'])
y = house_data['SALES_PRICE']

# Split the data into training and testing sets
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

#Train a model to assess feature importance
model = RandomForestRegressor(random_state=42)
model.fit(X_train,y_train)

# Feature importance analysis
importances = model.feature_importances_
feature_importance = pd.DataFrame({'Feature': X.columns,'Importance':importances})
print("Feature Importance:\n", feature_importance.sort_values(by = 'Importance',ascending=False))

# optiopnally: Drop features with very low importance if desired
threshold = 0.01
important_features = feature_importance[feature_importance['Importance'] > threshold]['Feature']
X = X[important_features.tolist()]

print("Feature Selection Completed. Selected Features Saved")

Feature Importance:
           Feature  Importance
12        REG_FEE    0.770318
0            AREA    0.075496
1        INT_SQFT    0.056212
7       BUILDTYPE    0.035159
3       N_BEDROOM    0.016153
13         COMMIS    0.009773
4      N_BATHROOM    0.009755
2   DIST_MAINROAD    0.003215
11     QS_OVERALL    0.002956
8        QS_ROOMS    0.002752
9     QS_BATHROOM    0.002731
6      PARK_FACIL    0.002578
10     QS_BEDROOM    0.002528
14      SALE_YEAR    0.002392
16     BUILD_YEAR    0.002285
17      BUILD_AGE    0.002255
15     SALE_MONTH    0.002069
5          N_ROOM    0.001372
Feature Selection Completed. Selected features Saved


Model Training For House Price Prediction

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso,Ridge
from sklearn.metrics import mean_squared_error,mean_absolute_error,r2_score
import joblib
import time

# Load the Data
file_path  = r"D:\\GUVI AI & ML\\Machine-Learning\\Streamlit_house_prediction\\preprocessed_house_data.csv"
house_data = pd.read_csv(file_path)

# Define features and target variable based on selected features

selected_features = [
    'AREA', 'INT_SQFT', 'DIST_MAINROAD', 'N_BEDROOM', 'N_BATHROOM',
    'N_ROOM', 'PARK_FACIL', 'BUILDTYPE', 'QS_ROOMS', 'QS_BATHROOM',
    'QS_BEDROOM', 'QS_OVERALL', 'REG_FEE', 'COMMIS',
    'SALE_YEAR', 'SALE_MONTH', 'BUILD_YEAR', 'BUILD_AGE'
]

X = house_data[selected_features]
y = house_data['SALES_PRICE']

# Split the data to training and testing sets

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# Define models and hyperparameters for grid search

models = {
    'RandomForest': RandomForestRegressor(random_state=42),
    'Lasso': Lasso(),
    'Ridge': Ridge()
}

param_grids = {
    'RandomForest':{
      'n_estimators':[100,200],
      'max_depth':[None,10,20],
      'min_samples_split': [2,5,10]
    },
    'Lasso':{
        'alpha':[0.01,0.1,1,10]
    },
    'Ridge':{
        'alpha':[0.01,0.1,1,10]
    }
}


# Perform grid search and Train models

best_models = {}
training_times = {}
prediction_times = {}

for model_name,model in models.items():
    print(f"Training {model_name}....")


    start_train_time = time.time()
    grid_search = GridSearchCV(model,param_grids[model_name],cv=5,scoring='neg_mean_squared_error')
    grid_search.fit(X_train,y_train)
    best_models[model_name] = grid_search.best_estimator_
    end_train_time = time.time()


    training_time = end_train_time -start_train_time
    training_times[model_name] = training_time

    # Evaluate the model
    start_pred_time = time.time()
    y_pred = best_models[model_name].predict(X_test)
    end_pred_time = time.time()

    prediction_time = end_pred_time - start_pred_time
    prediction_times[model_name] = prediction_time


    mse = mean_squared_error(y_test,y_pred)
    mae = mean_absolute_error(y_test,y_pred)
    r2 = r2_score(y_test,y_pred)


    print(f"{model_name} MSE: {mse}")
    print(f"{model_name} MAE: {mae}")
    print(f"{model_name} R2: {r2}")
    print(f"{model_name} Training Time: {training_time:.4f} seconds")
    print(f"{model_name} Prediction Time: {prediction_time:.4f} seconds")


# Save the best model(RandomForest in this case,assuming its the best)

best_model = best_models['RandomForest']

model_file_path = r"D:\\GUVI AI & ML\\Machine-Learning\\Streamlit_house_prediction\\best_model.pkl"
joblib.dump(best_model,model_file_path)

print("Best Model is Saved.")

Training RandomForest....
RandomForest MSE: 566697631357.9773
RandomForest MAE: 597703.539366232
RandomForest R2: 0.9568638013380142
RandomForest Training Time: 386.6271 seconds
RandomForest Prediction Time: 0.0527 seconds
Training Lasso....
Lasso MSE: 1904189296069.1023
Lasso MAE: 1104450.7312662797
Lasso R2: 0.8550558830316037
Lasso Training Time: 0.5752 seconds
Lasso Prediction Time: 0.0011 seconds
Training Ridge....
Ridge MSE: 1904217206801.2278
Ridge MAE: 1104459.3516337369
Ridge R2: 0.8550537585073087
Ridge Training Time: 0.0834 seconds
Ridge Prediction Time: 0.0010 seconds
Best Model is Saved.


d:\GUVI AI & ML\Machine-Learning\Streamlit_house_prediction\.venv\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.73456e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
d:\GUVI AI & ML\Machine-Learning\Streamlit_house_prediction\.venv\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.6071e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
d:\GUVI AI & ML\Machine-Learning\Streamlit_house_prediction\.venv\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.68839e-17): result may not be accurate.
  return linalg.solve(A, Xy, assume_a="pos", overwrite_a=True).T
d:\GUVI AI & ML\Machine-Learning\Streamlit_house_prediction\.venv\Lib\site-packages\sklearn\linear_model\_ridge.py:216: LinAlgWarning: Ill-conditioned matrix (rcond=7.66573e-17): result